# Prompts: Langchain

This notebook demonstrates how to use `PromptTemplate` from the `langchain_core` package. `PromptTemplate` tooling allows you to pre-design the form and content of text-based instructions sent to Language Models (LMs) during inference.

## Definitions

### OpenAI messages

**OpenAI conversations**
_definitions modified from: https://cdn.openai.com/spec/model-spec-2024-05-08.html_

**Assistant:** the entity that the end user or developer interacts with. While language models can generate text continuations of any input, the models we're using today have been fine-tuned on inputs formatted as conversations, consisting of a list of messages. In these conversations, the model is only designed to play one participant, called the assistant. In this document, when we discuss model behavior, we're referring to its behavior as the assistant; "model" and "assistant" will be approximately synonymous.

**Conversation:** valid input to the model is a conversation, which consists of a list of messages.

Example message:
```json
{
    "role": "assistant",
    "recipient": "browser",
    "content": "assist the user",
    "end_turn": true,
}
```

### A brief note about tokens

When you send the message to the LM, the request becomes a single string jammed with special tokens like so:
```
<|start|>assistant<|recipient|>browser<|content|>assist the user<|end_turn|>
```

where `<|...|>` denotes a special token.

### Claude conversations

_see https://docs.anthropic.com/claude/reference/messages_post for more information_

Example conversation:
```json
{
    "model": "claude-3-opus-20240229",
    "max_tokens": 1024,
    "system": "You are a helpful assistant...",
    "messages": [
        {"role": "user", "content": "Hello, world"}
    ]
}
```

## Prompts

In [1]:
from langchain_core.prompts import PromptTemplate

In [2]:
prompt_template = PromptTemplate.from_template(
    "{user} wants to hear a joke about: \"{content}\"."
)
prompt_template.format(user="Brendan", content="stage fright")

'Brendan wants to hear a joke about: "stage fright".'

In [3]:
# You don't need to use variables, this works just fine too:
prompt_template = PromptTemplate.from_template(
    "Tell me about your favorite library."
)
prompt_template.format()

'Tell me about your favorite library.'

In [4]:
from langchain_core.prompts import (
    ChatPromptTemplate,
    HumanMessagePromptTemplate
)
from langchain_core.messages import SystemMessage, HumanMessage

chat_template = ChatPromptTemplate.from_messages(
    [
        SystemMessage(
            content=(
                "You are a helpful assistant that provides information about "
                "library services and facilitates access to library resources."
            )
        ),
        HumanMessagePromptTemplate.from_template("{text}"),
    ]
)

chat_val = chat_template.invoke({"text": "How can I request books from other libraries besides this one?"})

In [5]:
chat_val.to_messages()

[SystemMessage(content='You are a helpful assistant that provides information about library services and facilitates access to library resources.'),
 HumanMessage(content='How can I request books from other libraries besides this one?')]

In [6]:
chat_val.to_string()

'System: You are a helpful assistant that provides information about library services and facilitates access to library resources.\nHuman: How can I request books from other libraries besides this one?'

## Advanced Prompting

In [7]:
from typing import List, Optional


def prompt_template() -> str:
    return """Please provide an answer to the question based on the documents provided. Include specific details from the documents that support your answer. Each document is identified by a 'title' and a unique 'source' UUID:

Documents:
{context}
Answer in raw markdown. When referencing a document by title, link to it using its UUID like this: [title](https://dc.library.northwestern.edu/items/UUID). For example: [Judy Collins, Jackson Hole Folk Festival](https://dc.library.northwestern.edu/items/f1ca513b-7d13-4af6-ad7b-8c7ffd1d3a37). Suggest keyword searches using this format: [keyword](https://dc.library.northwestern.edu/search?q=keyword). Offer a variety of search terms that cover different aspects of the topic. Include as many direct links to Digital Collections searches as necessary for a thorough study. The `collection` field contains information about the collection the document belongs to. In the summary, mention the top 1 or 2 collections, explain why they are relevant and link to them using the collection title and id: [collection['title']](https://dc.library.northwestern.edu/collections/collection['id']), for example [World War II Poster Collection](https://dc.library.northwestern.edu/collections/faf4f60e-78e0-4fbf-96ce-4ca8b4df597a):

Question:
{question}
"""


def document_template(attributes: Optional[List[str]] = None) -> str:
    if attributes is None:
        attributes = []
    lines = (
        ["Content: {title}", "Metadata:"]
        + [f"  {attribute}: {{{attribute}}}" for attribute in attributes]
        + ["Source: {id}"]
    )
    return "\n".join(lines)

In [8]:
context = document_template(attributes=["collection", "date", "description", "format", "language", "rights", "subject", "type"])
print(context)

Content: {title}
Metadata:
  collection: {collection}
  date: {date}
  description: {description}
  format: {format}
  language: {language}
  rights: {rights}
  subject: {subject}
  type: {type}
Source: {id}


In [9]:
print(prompt_template())

Please provide an answer to the question based on the documents provided. Include specific details from the documents that support your answer. Each document is identified by a 'title' and a unique 'source' UUID:

Documents:
{context}
Answer in raw markdown. When referencing a document by title, link to it using its UUID like this: [title](https://dc.library.northwestern.edu/items/UUID). For example: [Judy Collins, Jackson Hole Folk Festival](https://dc.library.northwestern.edu/items/f1ca513b-7d13-4af6-ad7b-8c7ffd1d3a37). Suggest keyword searches using this format: [keyword](https://dc.library.northwestern.edu/search?q=keyword). Offer a variety of search terms that cover different aspects of the topic. Include as many direct links to Digital Collections searches as necessary for a thorough study. The `collection` field contains information about the collection the document belongs to. In the summary, mention the top 1 or 2 collections, explain why they are relevant and link to them usi